## Vamos a importar aqui las librerías y documentos necesarios.

In [1]:
import os
os.chdir ('D:\Bootcamp_22\proyecto_twitter')
os.getcwd()
from Utils import credentials

In [2]:
import json
import tweepy
import requests
import pandas as pd
from datetime import datetime
import sqlite3

#### Ingresamos las Keys y los Token y contactamos con la API.

In [3]:
auth=tweepy.OAuthHandler(credentials.API_KEY,credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN,credentials.ACCESS_TOKEN_SECRET)

api=tweepy.API(auth)


In [4]:
#lista_mensajes={}
for message in tweepy.Cursor(api.search_full_archive,'development','@TheBridge_Tech').items(150):
#        lista_mensajes.append(message)
        print(json.dumps (message._json, indent=2))

TooManyRequests: 429 Too Many Requests
Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.

#### Con el siguiente código extraemos la información de la API y la guardo en un DataFrame.

El fichero solo contiene datos hasta principios desde Septiembre

In [4]:
db = pd.DataFrame(columns=['identity', 'text', 'created', 'user_id', 
                               'user_name', 'user_screen_name', 'quote', 'reply', 'retweet', 'favorite']) 
tweets= tweepy.Cursor(api.search_full_archive,'development','@TheBridge_Tech').items(250)
    
list_messages = [tweet for tweet in tweets] 
i = 1  
    
for tweet in list_messages: 
    identity = tweet._json["id"] 
    text = tweet._json["text"] 
    created = tweet._json["created_at"] 
    user_id = tweet._json["user"]["id"]
    user_name = tweet._json["user"]["name"] 
    user_screen_name = tweet._json["user"]["screen_name"] 
    quote = tweet._json["quote_count"] 
    reply = tweet._json["reply_count"] 
    retweet = tweet._json["retweet_count"]
    favorite = tweet._json["favorite_count"]

  


    ith_tweet = [identity, text, created, user_id, user_name, user_screen_name, quote, reply, retweet, favorite]
    print(ith_tweet)
    db.loc[len(db)] = ith_tweet
    i = i+1
db.head()

 
            

TooManyRequests: 429 Too Many Requests
Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.

In [8]:
db.head()

,identity,text,created,user_id,user_name,user_screen_name,quote,reply,retweet,favorite
0,1578246533832654848,RT @TheBridge_Tech: El próximo martes 11 de oc...,Fri Oct 07 04:51:10 +0000 2022,1099588676181471233,Flor_Turoninos,turoninos,0,0,0,0
1,1577720804619571201,RT @TheBridge_Tech: Ya están aquí nuestros pri...,Wed Oct 05 18:02:07 +0000 2022,1099588676181471233,Flor_Turoninos,turoninos,0,0,0,0
2,1577648996927012866,RT @TheBridge_Tech: ¡Feliz Día a nuestros Lead...,Wed Oct 05 13:16:46 +0000 2022,763045308095950848,Carlos Sevilla,Carlosevi94,0,0,0,0
3,1577646040269864960,RT @TheBridge_Tech: ¡Feliz Día a nuestros Lead...,Wed Oct 05 13:05:01 +0000 2022,1099588676181471233,Flor_Turoninos,turoninos,0,0,0,0
4,1577335173523308544,RT @PauMugarra: Recordaros que la semana que v...,Tue Oct 04 16:29:45 +0000 2022,7154412,Javier G. Recuenco,Recuenco,0,0,0,0


#### Convertimos la columna *created* en fecha y ajustamos los datos obtenidos a las condiciones de las fechas indicadas en el caso.

In [9]:
db['created'] = pd.to_datetime(db['created'])

In [10]:
db = db.drop(db[db['created']>'2022-10-05 18:02:07+00:00'].index)

In [11]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 1 to 60
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   identity          60 non-null     int64              
 1   text              60 non-null     object             
 2   created           60 non-null     datetime64[ns, UTC]
 3   user_id           60 non-null     int64              
 4   user_name         60 non-null     object             
 5   user_screen_name  60 non-null     object             
 6   quote             60 non-null     int64              
 7   reply             60 non-null     int64              
 8   retweet           60 non-null     int64              
 9   favorite          60 non-null     int64              
dtypes: datetime64[ns, UTC](1), int64(6), object(3)
memory usage: 5.2+ KB


## Generamos el DataFrame de users para generar una tabla en SQL.

In [244]:
df_users=pd.DataFrame()

In [245]:
df_users['user_id']=db['user_id']
df_users['user_name']=db['user_id']
df_users['user_screen_name']=db['user_screen_name']
df_users.head()


,user_id,user_name,user_screen_name
2,1099588676181471233,1099588676181471233,turoninos
3,763045308095950848,763045308095950848,Carlosevi94
4,1099588676181471233,1099588676181471233,turoninos
5,7154412,7154412,Recuenco
6,1003872445,1003872445,PauMugarra


In [246]:
df_users = df_users.drop_duplicates(['user_name'],keep='last')

In [247]:
df_users['user_name'].value_counts(ascending=True)

763045308095950848     1
2227356042             1
1131933723681800193    1
41110484               1
210026854              1
1051649960             1
1073622740             1
1155468348139982848    1
2390191837             1
394734366              1
1327588137346998273    1
1398866593237508100    1
1533105519447404544    1
1641440478             1
1565028593629003777    1
1099588676181471233    1
1367036537943244803    1
1494355208473874438    1
2907682397             1
581925228              1
12085582               1
992943418052460544     1
7154412                1
1392422091434168323    1
1157259865389244417    1
2529499620             1
706520411551494145     1
21478536               1
917321990465351681     1
127000294              1
604485175              1
1391838162830385165    1
1127630178644054018    1
568101757              1
1262459681693409281    1
1393908772754575360    1
1252305008357527553    1
1003872445             1
975452621522853890     1
1311739853307027457    1


In [254]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 3 to 69
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           41 non-null     int64 
 1   user_name         41 non-null     int64 
 2   user_screen_name  41 non-null     object
dtypes: int64(2), object(1)
memory usage: 1.3+ KB


## Generamos otro Dataframe con los Tweets.

Eliminamos los registros de _user_name_ y _user_screen_name_ del Dataframe, ya que se vincularán con la Tabla de users en SQL a través de una Query.


In [248]:
df_tweets=db.drop(columns=['user_name','user_screen_name'])
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 2 to 69
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   identity  68 non-null     int64              
 1   text      68 non-null     object             
 2   created   68 non-null     datetime64[ns, UTC]
 3   user_id   68 non-null     int64              
 4   quote     68 non-null     int64              
 5   reply     68 non-null     int64              
 6   retweet   68 non-null     int64              
 7   favorite  68 non-null     int64              
dtypes: datetime64[ns, UTC](1), int64(6), object(1)
memory usage: 4.8+ KB


In [249]:
df_tweets.head()

,identity,text,created,user_id,quote,reply,retweet,favorite
2,1577720804619571201,RT @TheBridge_Tech: Ya están aquí nuestros pri...,2022-10-05 18:02:07+00:00,1099588676181471233,0,0,0,0
3,1577648996927012866,RT @TheBridge_Tech: ¡Feliz Día a nuestros Lead...,2022-10-05 13:16:46+00:00,763045308095950848,0,0,0,0
4,1577646040269864960,RT @TheBridge_Tech: ¡Feliz Día a nuestros Lead...,2022-10-05 13:05:01+00:00,1099588676181471233,0,0,0,0
5,1577335173523308544,RT @PauMugarra: Recordaros que la semana que v...,2022-10-04 16:29:45+00:00,7154412,0,0,0,0
6,1577334577701453827,Recordaros que la semana que viene tenemos la ...,2022-10-04 16:27:23+00:00,1003872445,0,0,2,12


## Generamos las tablas de SQL.

In [253]:
connection = sqlite3.connect("Data/Twitter_bridge.db")
crsr = connection.cursor()
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

USERS
TWEETS


In [250]:
query = '''
CREATE TABLE USERS (
    user_id INT NOT NULL,
    user_name VARCHAR(50) NOT NULL,
    user_screen_name VARCHAR(50) NOT NULL,
    PRIMARY KEY (user_id)   
);'''
crsr.execute(query)

In [252]:
query = '''
CREATE TABLE TWEETS (
    identity INT NOT NULL,
    text VARCHAR(250) NOT NULL,
    created DATE NOT NULL,
    user_id INT NOT NULL,
    quote INT, 
    reply INT,
    retweet INT,
    favorite INT,
	FOREIGN KEY (user_id) REFERENCES USERS(user_id),
    PRIMARY KEY (identity)   
);'''
crsr.execute(query)

#### Ingresamos los datos desde los DataFrames a las tablas generadas en SQL.

In [257]:
df_users.to_sql("USERS", connection, if_exists="replace")

41

In [258]:
df_tweets.to_sql("TWEETS", connection, if_exists="replace")

68

In [ ]:
# ATENCIÓN! Al terminar hay que cerrar la conexión a la DB para que se guarden los cambios!!!!!!
connection.commit()
connection.close()